## Project Summary: 

The project investigates the relationship between various socioeconomic factors - such as poverty, age distribution, cost of living, racial and ethnic minority status, and others - and the distribution of married vs. unmarried couples across counties in the United States. The analysis uses 2020 data from the Social Vulnerability Index and five years of household data from the American Community Survey.

To explore this relationship, the student first applies clustering based on five key features, one of which is the slope of the best-fit curve to household data. This slope acts as a filtering feature, where polynomials with an $R^2 < 0.6$ are considered noisy and removed from the dataset.

Next, the student uses a random forest model to predict the cluster classes. The model’s performance is assessed using cross-validation. The prediction error is slightly above the random guessing error rate, indicating that the model has reasonable predictive power.

## Non-technical improvement 

The student does not provide graphs to illustrate the results of the cross-validation of the random forest model. Visual representations would have made it easier to understand the results and the conclusions drawn from them. Additionally, graphs highlighting the important features would have strengthened the argument and supported the conclusions more effectively. Furthermore, it would have been helpful if the student added some examples of the raw data and the data after computing and normalizing the features. 

## Technical improvement

While the student mentioned that they normalized the features (with the exception for the best fit curve, which is binary), I believe that they did not take into account the percentage change of population, rather, they took the absolute change. If I understand the code correctly, right now the "slope" and "acceleration" features are raw counts of households per year, so two counties with the same percentage growth but different base populations end up too far away. In order to fix this issue, I recommend looking at the percentage change. 

In [1]:
from itertools import product
import csv
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# You can also obtain this via CourseWorks
df = pd.read_csv(
    '/Users/EleanorSchwartz/Desktop/Spring 2025/Applied Machine Learning/hw5/with_geo_household_cnt.csv',
    usecols=['NAME', 'state', 'county', 'INTPTLAT', 'INTPTLON',
             'B11002_003E', 'B11002_012E', 'year'])

In [18]:
def fit_best_polynomial(X, Y, k=1):
    X_powers = X.copy()
    for i in range(2, k+1):
        X_powers = np.concatenate([X_powers, np.power(X, i)], axis=1)
    assert X_powers.shape[1] == k
    mod = LinearRegression().fit(X_powers, Y)
    return np.concatenate([mod.intercept_,
                           mod.coef_[0],
                           np.array([mod.score(X_powers, Y)])]) # R^2


def get_best_curve(sdf, census_var='B11002_003E'):
    X = sdf.year.to_numpy().reshape(-1, 1)
    Y = sdf[census_var].to_numpy().reshape(-1, 1)
    poly_stats = []
    for p in range(1, 4):
        poly_stats.append(fit_best_polynomial(X, Y, p))
    poly_stat_all = np.concatenate(poly_stats)
    return poly_stat_all


# Use smallest polynomial that passes 0.6 R^2 value
# calculate slope at 2023/2024
# calculate acceleration at 2023/2024
# calculate if there was a slope change ever
# if no best line fit exists, separate it out

def calc_slope(coefs, x):
    # assumes getting a linear model
    slope = 0 * x
    for j, coef in enumerate(coefs):
        # first term will always be 0
        comp = j * coef * np.power(x, float(j - 1))
        slope = slope + comp
    return slope


def get_feats(row, r2_cutoff=0.6):
    r2_inds = [2, 6, 11]
    years = np.array(range(2009, 2024))
    if all(row[r2_inds] < r2_cutoff):
        return np.array([np.nan] * 5)
    for p, r2_i in enumerate(r2_inds):
        if row[r2_i] < r2_cutoff:
            continue
        coefs_start = p if p == 0 else r2_inds[p - 1] + 1
        p += 1
        coefs = row[coefs_start:r2_i]
        slope = calc_slope(coefs, years)
        # acc is slope of the slope
        acc = calc_slope([c * i for i, c in enumerate(coefs) if i > 0],
                         years)
        steady_slope = np.array([1 if all(slope > 0) or all(slope < 0) else 0])

        return np.concatenate([slope[-2:], acc[-2:], steady_slope], axis=0)


df_grp = df.groupby(['NAME', 'state', 'county'])

ts_fits = []
names = []
census_vars = ['B11002_003E', 'B11002_012E']
# grp, ind = next(iter(df_grp.groups.items()))
for grp, ind in df_grp.groups.items():
    names.append(grp[0])
    sdf = df.loc[ind, ].copy()
    is_22 = sdf.year == 2022
    best_curves = [np.concatenate([get_best_curve(sdf, cv),
                                   sdf.loc[is_22, cv].to_numpy()]) for cv in census_vars]
    curve_feats = [np.concatenate([get_feats(bc), bc[-1:]]) for bc in best_curves]
    ts_fits.append(np.concatenate(curve_feats))

ts_df = pd.DataFrame(ts_fits)
ts_df.index = names
col_names = ['slope_2022', 'slope_2023', 'acc_2022', 'acc_2023', 'steady_slope', 'val_2022']
df_col_names = [status + '-' + col for status, col in product(['married', 'unmarried'], col_names)]
ts_df.columns = df_col_names

for status in ['married', 'unmarried']:
    base_col = f"{status}-val_2022"  
    print(f'{status} example base case {ts_df[base_col][3]}')
    ts_df[f"{status}-pct_slope_2022"] = ts_df[f"{status}-slope_2022"] / ts_df[base_col]
    print(f'{status} example percentage slope {ts_df[f"{status}-pct_slope_2022"][3]}')
    ts_df[f"{status}-pct_slope_2023"] = ts_df[f"{status}-slope_2023"] / ts_df[base_col]
    print(f'{status} example percentage slope {ts_df[f"{status}-pct_slope_2023"][3]}')
    ts_df[f"{status}-pct_acc_2022"] = ts_df[f"{status}-acc_2022"] / ts_df[base_col]
    print(f'{status} example percentage acceleration {ts_df[f"{status}-pct_acc_2022"][3]}')
    ts_df[f"{status}-pct_acc_2023"] = ts_df[f"{status}-acc_2023"] / ts_df[base_col]
    print(f'{status} example percentage acceleration {ts_df[f"{status}-pct_acc_2023"][3]}')

ts_df.to_csv('curve_feats_counties.csv', quoting=csv.QUOTE_NONNUMERIC)

married example base case 319232.0
married example percentage slope 0.017515351575925077
married example percentage slope 0.017515351575925077
married example percentage acceleration 0.0
married example percentage acceleration 0.0
unmarried example base case 88463.0
unmarried example percentage slope 0.025645346480610946
unmarried example percentage slope 0.025645346480610946
unmarried example percentage acceleration 0.0
unmarried example percentage acceleration 0.0


In [13]:
ts_df.head()

,married-slope_2022,married-slope_2023,married-acc_2022,married-acc_2023,married-steady_slope,married-val_2022,unmarried-slope_2022,unmarried-slope_2023,unmarried-acc_2022,unmarried-acc_2023,unmarried-steady_slope,unmarried-val_2022,married-pct_slope_2022,married-pct_slope_2023,married-pct_acc_2022,married-pct_acc_2023,unmarried-pct_slope_2022,unmarried-pct_slope_2023,unmarried-pct_acc_2022,unmarried-pct_acc_2023
"Abbeville County, South Carolina",-164.900000,-164.900000,0.000000,0.000000,1.0,13492.0,NaN,NaN,NaN,NaN,NaN,3971.0,-0.012222,-0.012222,0.000000,0.000000,NaN,NaN,NaN,NaN
"Acadia Parish, Louisiana",-1780.133904,-2475.779204,-647.019608,-744.270990,0.0,34432.0,-275.461167,-337.813623,-62.352456,-62.352456,0.0,7885.0,-0.051700,-0.071903,-0.018791,-0.021616,-0.034935,-0.042843,-0.007908,-0.007908
"Accomack County, Virginia",-150.004223,-263.183387,-101.208953,-125.149375,0.0,18488.0,-45.042857,-45.042857,0.000000,0.000000,1.0,5339.0,-0.008114,-0.014235,-0.005474,-0.006769,-0.008437,-0.008437,0.000000,0.000000
"Ada County, Idaho",5591.460714,5591.460714,0.000000,0.000000,1.0,319232.0,2268.664286,2268.664286,0.000000,0.000000,1.0,88463.0,0.017515,0.017515,0.000000,0.000000,0.025645,0.025645,0.000000,0.000000
"Adair County, Iowa",-75.853571,-75.853571,0.000000,0.000000,1.0,4560.0,142.008268,192.737933,47.495314,53.964016,0.0,1624.0,-0.016635,-0.016635,0.000000,0.000000,0.087444,0.118681,0.029246,0.033229
